# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import time
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dalam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dalam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dalam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM df', con=engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26216 non-null  int64 
 1   message                 26216 non-null  object
 2   original                10170 non-null  object
 3   genre                   26216 non-null  object
 4   related                 26216 non-null  int64 
 5   request                 26216 non-null  int64 
 6   offer                   26216 non-null  int64 
 7   aid_related             26216 non-null  int64 
 8   medical_help            26216 non-null  int64 
 9   medical_products        26216 non-null  int64 
 10  search_and_rescue       26216 non-null  int64 
 11  security                26216 non-null  int64 
 12  military                26216 non-null  int64 
 13  child_alone             26216 non-null  int64 
 14  water                   26216 non-null  int64 
 15  fo

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):

    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()

    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # tokenize text
    tokens = word_tokenize(text)

    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word, pos='v')
              for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline_AdaBoost = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
start = time.perf_counter()
pipeline_AdaBoost.fit(X_train, y_train)
y_pred_AdaBoost = pipeline_AdaBoost.predict(X_test)
end = time.perf_counter()
print(f"Time spent to run the AdaBoost pipeline is {end - start:0.4f} seconds")

Time spent to run the AdaBoost pipeline is 128.3545 seconds


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
print(classification_report(y_test, y_pred_AdaBoost,
                            target_names=y_test.columns.values))

precision    recall  f1-score   support

               related       0.80      0.96      0.87      5002
               request       0.79      0.49      0.61      1164
                 offer       0.00      0.00      0.00        20
           aid_related       0.74      0.60      0.66      2723
          medical_help       0.58      0.28      0.38       506
      medical_products       0.67      0.30      0.41       321
     search_and_rescue       0.53      0.17      0.26       171
              security       0.23      0.04      0.07       125
              military       0.64      0.35      0.45       219
           child_alone       0.00      0.00      0.00         0
                 water       0.72      0.67      0.69       393
                  food       0.82      0.63      0.71       736
               shelter       0.74      0.53      0.61       557
              clothing       0.74      0.46      0.57        98
                 money       0.49      0.29      0.36       148

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline_AdaBoost.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000025787EDA318>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000025787EDA318>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [10]:
parameters_AdaBoost = {'clf__estimator__learning_rate': [0.5, 0.75, 1],
                       'clf__estimator__n_estimators': [25, 50, 75, 100]}

cv_AdaBoost = GridSearchCV(pipeline_AdaBoost,
                           param_grid=parameters_AdaBoost, scoring='f1_micro', n_jobs=-1)

In [11]:
start = time.perf_counter()
cv_AdaBoost.fit(X_train, y_train)
y_pred_cv_AdaBoost = cv_AdaBoost.predict(X_test)
end = time.perf_counter()
print(f"Time spent to run the AdaBoost GridSearch is {end - start:0.4f} seconds")

Time spent to run the AdaBoost GridSearch is 3685.7062 seconds


In [12]:
print("\nBest Parameters:", cv_AdaBoost.best_params_)


Best Parameters: {'clf__estimator__learning_rate': 1, 'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
print(classification_report(y_test, y_pred_cv_AdaBoost,
                            target_names=y_test.columns.values))

precision    recall  f1-score   support

               related       0.81      0.96      0.88      5002
               request       0.79      0.53      0.63      1164
                 offer       0.00      0.00      0.00        20
           aid_related       0.76      0.61      0.68      2723
          medical_help       0.54      0.27      0.36       506
      medical_products       0.59      0.31      0.40       321
     search_and_rescue       0.39      0.15      0.22       171
              security       0.23      0.06      0.09       125
              military       0.61      0.35      0.44       219
           child_alone       0.00      0.00      0.00         0
                 water       0.69      0.67      0.68       393
                  food       0.80      0.62      0.70       736
               shelter       0.72      0.50      0.59       557
              clothing       0.76      0.46      0.57        98
                 money       0.46      0.32      0.38       148

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
pipeline_RandomForest = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

In [15]:
start = time.perf_counter()
pipeline_RandomForest.fit(X_train, y_train)
y_pred_RandomForest = pipeline_RandomForest.predict(X_test)
end = time.perf_counter()
print(f"Time spent to run the RandomForest pipeline is {end - start:0.4f} seconds")

Time spent to run the RandomForest pipeline is 640.0503 seconds


In [16]:
print(classification_report(y_test, y_pred_RandomForest,
                            target_names=y_test.columns.values))

precision    recall  f1-score   support

               related       0.85      0.93      0.89      5002
               request       0.85      0.48      0.61      1164
                 offer       0.00      0.00      0.00        20
           aid_related       0.76      0.68      0.72      2723
          medical_help       0.63      0.07      0.13       506
      medical_products       0.89      0.10      0.17       321
     search_and_rescue       0.43      0.02      0.03       171
              security       0.00      0.00      0.00       125
              military       0.67      0.05      0.09       219
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.38      0.53       393
                  food       0.86      0.53      0.65       736
               shelter       0.81      0.38      0.52       557
              clothing       0.91      0.10      0.18        98
                 money       0.60      0.04      0.08       148

In [17]:
pipeline_RandomForest.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000025787EDA318>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000025787EDA318>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [18]:
parameters_RandomForest = {'clf__estimator__max_features': ['auto'],
                           'clf__estimator__n_estimators': [25, 50, 75, 100]}

cv_RandomForest = GridSearchCV(
    pipeline_RandomForest, param_grid=parameters_RandomForest, scoring='f1_micro', n_jobs=-1)

In [19]:
start = time.perf_counter()
cv_RandomForest.fit(X_train, y_train)
y_pred_cv_RandomForest = cv_RandomForest.predict(X_test)
end = time.perf_counter()
print(f"Time spent to run the RandomForest GridSearch is {end - start:0.4f} seconds")

Time spent to run the RandomForest GridSearch is 2691.0438 seconds


In [20]:
print("\nBest Parameters:", cv_RandomForest.best_params_)


Best Parameters: {'clf__estimator__max_features': 'auto', 'clf__estimator__n_estimators': 75}


In [21]:
print(classification_report(y_test, y_pred_cv_RandomForest,
                            target_names=y_test.columns.values))

precision    recall  f1-score   support

               related       0.85      0.93      0.89      5002
               request       0.85      0.48      0.62      1164
                 offer       0.00      0.00      0.00        20
           aid_related       0.77      0.67      0.72      2723
          medical_help       0.55      0.08      0.14       506
      medical_products       0.92      0.07      0.13       321
     search_and_rescue       0.54      0.04      0.08       171
              security       0.50      0.01      0.02       125
              military       0.70      0.09      0.15       219
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.40      0.54       393
                  food       0.86      0.60      0.70       736
               shelter       0.81      0.38      0.52       557
              clothing       0.82      0.09      0.17        98
                 money       0.60      0.04      0.08       148

### 9. Export your model as a pickle file

In [22]:
# According to the f1-score results, the best model is:
print("\nBest Estimator:", cv_AdaBoost.best_estimator_)


Best Estimator: Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000025787EDA318>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=1,
                                                                    n_estimators=100)))])


In [23]:
outfile = open('classifier.pickle', 'wb')
pickle.dump(cv_AdaBoost.best_estimator_, outfile)
outfile.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.